In [36]:
from metapub import PubMedFetcher
from metapub import FindIt
from datetime import datetime, timedelta
from deepl_conect import Translate

### App parameters

In [37]:
fetch = PubMedFetcher()

In [38]:
end_date = datetime.now()
start_date = end_date-timedelta(days=7)

In [39]:
start_date_str = start_date.strftime('%Y/%m/%d')
end_date_str = end_date.strftime('%Y/%m/%d')

### User parameters

Query

In [40]:
query = 'macitentan'

Criterios de seleccion
- Query en el titulo (+1)
- Research clinical trial (rct) en cualquier sitio (+2)
- Hecho en España (+2)
- Meta-analysis en cualquier sitio (+2)
- Es europeo/americano/australiano (+1)   

In [41]:
countries = ['usa','canada','italy','france','denmark','austria','belgium','finland','germany','greece','ireland','netherlands','norway','portugal','sweden','switzerland','uk', 'states']
query_in_title = 1
is_rct = 3
made_in_spain = 3
is_meta_analysis = 2
from_countries = 1
threshold = 2

Search functions

In [42]:
pmids = fetch.pmids_for_query(f'({query}) AND (("{start_date_str}"[Date - Publication] : "{end_date_str}"[Date - Publication]))')

In [43]:
print(f'({query}) AND (("{start_date_str}"[Date - Publication] : "{end_date_str}"[Date - Publication]))')

(macitentan) AND (("2024/03/01"[Date - Publication] : "2024/03/08"[Date - Publication]))


In [44]:
len(pmids)

3

Classification functions

In [45]:
article = fetch.article_by_pmid(str(pmids[0]))

In [46]:
def get_affiliations(art):
    article_dic = art.to_dict()
    affiliations_set = set()
    for auth in article_dic['author_list']:
        if len(auth.affiliations) != 0:
            affiliations_set.add(auth.affiliations[0].split()[-1].strip('.').lower())
    return affiliations_set

In [47]:
affiliations_set = get_affiliations(article)
affiliations_set

{'rchannick@mednet.ucla.edu', 'switzerland', 'usa'}

In [48]:
def from_countries_func(aff_set):
    for country in aff_set:
        if country in countries:
            return True
    return False

In [49]:
from_countries_func(affiliations_set)

True

In [50]:
def made_in_spain_func(aff_set):
    return True if 'spain' in countries else False
        

In [51]:
made_in_spain_func(affiliations_set)

False

In [52]:
def query_in_title_func(art):
    return True if query.lower() in art.title.lower() else False

In [53]:
query_in_title_func(article)

True

In [54]:
def is_rct_func(art):
    if art.abstract != None:
        there_is_rct = 'rct' in art.abstract.lower()
        there_is_research = 'research clinical trial' in art.abstract.lower()
        return True if (there_is_rct or there_is_research) else False

In [55]:
is_rct_func(article)

False

In [56]:
def is_meta_analysis_func(art):
    if article.abstract != None:
        in_abstract = 'meta-analyis' in article.abstract.lower() 
        in_title = 'meta-analyis' in article.title.lower()
        return True if (in_abstract or in_title) else False

In [57]:
is_meta_analysis_func(article)

False

Rater function

In [64]:
def rater(art):
    affiliations_set = get_affiliations(art)
    score = 0
    tresh = threshold
    if query_in_title_func(art):
        score = query_in_title
    if from_countries_func(affiliations_set):
        score += from_countries
    if is_rct_func(art):
        score += is_rct
    if made_in_spain_func(affiliations_set):
        score += made_in_spain
    if is_meta_analysis_func(art):
        score += is_meta_analysis
    print(score)
    if score >= threshold:
        return True
        

In [65]:
rater(article)

2


True

In [66]:
selected = list()
for pmid in pmids:
    article = fetch.article_by_pmid(str(pmid))
    if rater(article):
        selected.append(article)


2
2
2


In [67]:
len(selected)

3

### Display information

In [62]:
transl = Translate()

In [68]:
for art in selected:
    print(art.title)
    print(art.authors_str)
    print(f'https://doi.org/{art.doi}')
    if art.abstract != None:
        print(transl.translate_to_sp(art.abstract))
    src = FindIt(str(art.pmid))
    if src.url:
        print(src.url)
    else:
        print('No open access')
        print(src.reason)
    print('\n')

2024-03-08 16:06:09 MSI-Jose deepl[21684] INFO Request to DeepL API method=POST url=https://api-free.deepl.com/v2/translate


Macitentan in Pulmonary Arterial Hypertension Associated with Connective Tissue Disease (CTD-PAH): Real-World Evidence from the Combined OPUS/OrPHeUS Dataset.
Channick R; Chin KM; McLaughlin VV; Lammi MR; Zamanian RT; Turricchia S; Ong R; Mitchell L; Kim NH
https://doi.org/10.1007/s40119-024-00361-w


2024-03-08 16:06:09 MSI-Jose deepl[21684] INFO DeepL API response status_code=200 url=https://api-free.deepl.com/v2/translate


INTRODUCCIÓN: Los datos sobre la práctica clínica real y los resultados de los pacientes con hipertensión arterial pulmonar asociada a enfermedad del tejido conectivo (HAP-TCP) son escasos. Los estudios OPUS/OrPHeUS incluyeron a pacientes que empezaban a recibir macitentan, incluidos los que padecían HAP-CTD. Este análisis describe las características de los pacientes, los patrones de tratamiento, los resultados y los perfiles de seguridad de los pacientes con CTD-PAH que iniciaron macitentan por primera vez en los EE.UU. utilizando el conjunto de datos combinados OPUS/OrPHeUS.
MÉTODOS: OPUS fue un registro de fármacos prospectivo, multicéntrico, observacional y a largo plazo de EE. UU. (abril de 2014-junio de 2020). OrPHeUS fue una revisión de historias clínicas retrospectiva, multicéntrica, estadounidense (octubre de 2013-marzo de 2017). Las características, los patrones de tratamiento, la seguridad y los resultados durante el tratamiento con macitentan de pacientes con CTD-PAH y sus

2024-03-08 16:06:10 MSI-Jose deepl[21684] INFO Request to DeepL API method=POST url=https://api-free.deepl.com/v2/translate


...

INTRODUCCIÓN: El objetivo de este subestudio fue evaluar la relación entre las variables de la ecocardiografía (eco) y la resonancia magnética cardiaca (RMc) y utilizar la eco para evaluar el efecto del macitentan en la estructura y función del ventrículo derecho (VD).
MÉTODOS: REPAIR (NCT02310672) fue un estudio prospectivo, multicéntrico, de un solo brazo, abierto, de 52 semanas, de fase 4 en pacientes con hipertensión arterial pulmonar (HAP), que investigó el efecto de macitentan 10 mg como monoterapia, o en combinación con un inhibidor de la fosfodiesterasa 5, sobre la estructura, función y hemodinámica del VD utilizando RMNc y cateterismo cardíaco derecho. En este subestudio, los pacientes también fueron evaluados mediante ecografía en el momento del cribado y en las semanas 26 y/o 52. Se realizaron análisis de correlación post hoc entre las variables de eco e IRMc mediante el coeficiente de correlación de Pearson, el coeficiente de correlación de Spearman y análisis de Bland-Altm

2024-03-08 16:06:15 MSI-Jose deepl[21684] INFO Request to DeepL API method=POST url=https://api-free.deepl.com/v2/translate


http://europepmc.org/backend/ptpmcrender.fcgi?accid=PMC10899124&blobtype=pdf


Safety of Macitentan for the Treatment of Portopulmonary Hypertension: Real-World Evidence from the Combined OPUS/OrPHeUS Studies.
Kim NH; Chin KM; McLaughlin VV; DuBrock H; Restrepo-Jaramillo R; Safdar Z; MacDonald G; Martin N; Rosenberg D; Solonets M; Channick R
https://doi.org/10.1007/s41030-023-00251-x


2024-03-08 16:06:16 MSI-Jose deepl[21684] INFO DeepL API response status_code=200 url=https://api-free.deepl.com/v2/translate


INTRODUCCIÓN: La hipertensión portopulmonar (HPP) conlleva un pronóstico peor que otras formas de hipertensión arterial pulmonar (HAP). Los datos relativos al uso de terapias específicas para la HAP en pacientes con HPP son escasos, ya que suelen quedar excluidos de los ensayos clínicos. Este análisis describe las características de los pacientes, los patrones de tratamiento, los resultados y los perfiles de seguridad en pacientes con HAPP que iniciaron macitentan recientemente en los EE.UU. utilizando el conjunto de datos combinados OPUS/OrPHeUS.
MÉTODOS: OPUS fue un registro de fármacos prospectivo, multicéntrico y observacional de EE. UU. (abril de 2014 a junio de 2020); OrPHeUS fue una revisión retrospectiva, multicéntrica y de gráficos de EE. UU. (octubre de 2013 a marzo de 2017). Se recopiló retrospectivamente información adicional sobre la enfermedad hepática de los pacientes con PoPH en OPUS.
RESULTADOS: El conjunto de datos OPUS/OrPHeUS incluyó 206 pacientes con PoPH (mediana 